In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import ast
from online.envs.bidding_env import BiddingEnv
from definitions import ROOT_DIR


In [2]:
from bidding_train_env.strategy import PlayerBiddingStrategy

In [3]:
strategy = PlayerBiddingStrategy()

env path /home/ubuntu/Dev/NeurIPS_Auto_Bidding_General_Track_Baseline/saved_model/PPO/033_ppo_seed_0_dense_base_ranges_29_obs_exp_multi_action/rl_model_vecnormalize_3500000_steps.pkl


/home/ubuntu/miniforge3/envs/alibaba/lib/python3.9/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment BiddingEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [59]:
action = np.random.randn(6) * 0.1
pvalues = np.random.uniform(low=0, high=0.01, size=3)
pvalues_sigmas = np.random.uniform(low=0, high=0.01, size=3)

print(pvalues)
coefs = strategy.train_env.compute_bid_coef(action, pvalues, pvalues_sigmas)
print(coefs)

[9.88351357e-05 8.00312323e-03 6.15944742e-04]
[0.         0.00437156 0.        ]


In [68]:
strategy.train_env.obs_keys

/home/ubuntu/miniforge3/envs/alibaba/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.obs_keys to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.obs_keys` for environment variables or `env.get_wrapper_attr('obs_keys')` that will search the reminding wrappers.
  logger.warn(


['time_left',
 'budget_left',
 'budget',
 'cpa',
 'category',
 'historical_bid_mean',
 'last_three_bid_mean',
 'least_winning_cost_mean',
 'last_least_winning_cost_mean',
 'last_three_least_winning_cost_mean',
 'pvalues_mean',
 'conversion_mean',
 'bid_success_mean',
 'last_pvalues_mean',
 'last_three_pvalues_mean',
 'last_conversion_mean',
 'last_three_conversion_mean',
 'last_bid_success',
 'last_three_bid_success_mean',
 'current_pvalues_mean',
 'current_pv_num',
 'last_three_pv_num',
 'pv_num_total',
 'historical_bid_over_lwc_mean',
 'last_bid_over_lwc_mean',
 'last_three_bid_over_lwc_mean',
 'historical_pv_over_lwc_mean',
 'last_pv_over_lwc_mean',
 'last_three_pv_over_lwc_mean']

In [2]:
train_data_path = (
    ROOT_DIR / "data/traffic_top_regression/training_data_16/training_data_all-rlData.csv"
)
training_data = pd.read_csv(train_data_path)

In [3]:
def safe_literal_eval(val):
    if pd.isna(val):
        return val  # 如果是NaN，返回NaN
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        print(ValueError)
        return val 

In [4]:
training_data["state"] = training_data["state"].apply(lambda x: np.array(safe_literal_eval(x)))
training_data["next_state"] = training_data["next_state"].apply(lambda x: np.array(safe_literal_eval(x)))
training_data.head()

,deliveryPeriodIndex,advertiserNumber,advertiserCategoryIndex,budget,CPAConstraint,timeStepIndex,state,action,reward,reward_continuous,done,next_state
0,7.0,0.0,0.0,750.0,8.0,0.0,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",15.000000,8.581538,9.442710,0,"[0.9791666666666666, 0.65502864, 0.09134144335..."
1,7.0,0.0,0.0,750.0,8.0,1.0,"[0.9791666666666666, 0.65502864, 0.09134144335...",17.999998,0.943003,1.923260,0,"[0.9583333333333334, 0.5264538, 0.101064443588..."
2,7.0,0.0,0.0,750.0,8.0,2.0,"[0.9583333333333334, 0.5264538, 0.101064443588...",12.599999,0.526889,0.559534,0,"[0.9375, 0.5100286, 0.09262432406346004, 0.092..."
3,7.0,0.0,0.0,750.0,8.0,3.0,"[0.9375, 0.5100286, 0.09262432406346004, 0.092...",8.820001,0.000000,0.012200,0,"[0.9166666666666666, 0.50986165, 0.08278862666..."
4,7.0,0.0,0.0,750.0,8.0,4.0,"[0.9166666666666666, 0.50986165, 0.08278862666...",10.584000,0.000000,0.235186,0,"[0.8958333333333334, 0.50575984, 0.07883444949..."


In [5]:
training_data.advertiserNumber.unique()

array([ 0.,  7.,  8., 12., 13., 16., 17., 20., 22., 26., 30., 32., 33.,
       34., 38., 39., 45., 46., 47.,  5.,  9., 21., 28., 37., 41., 43.,
        2.,  3., 23., 24., 31., 35., 40., 44., 19., 42., 10., 14., 15.,
       36.,  6.,  1.,  4., 27., 29.])

In [6]:
pvalues_path = ROOT_DIR / "data" / "online_rl_data" / "period-7_pvalues.parquet"
bids_path = ROOT_DIR / "data" / "online_rl_data" / "period-7_bids.parquet"

budget_range = (500, 3000)
target_cpa_range = (10, 12)
seed = 42

keys_list = [
    "time_left",
    "budget_left",
    "budget",
    "cpa",
    "category",
    "historical_bid_mean",
    "last_bid_mean",
    "last_three_bid_mean",
    "least_winning_cost_mean",
    "last_least_winning_cost_mean",
    "last_three_least_winning_cost_mean",
    "least_winning_cost_10_pct",
    "last_least_winning_cost_10_pct",
    "last_three_least_winning_cost_10_pct",
    "least_winning_cost_01_pct",
    "last_least_winning_cost_01_pct",
    "last_three_least_winning_cost_01_pct",
    "pvalues_mean",
    "conversion_mean",
    "bid_success_mean",
    "last_pvalues_mean",
    "last_three_pvalues_mean",
    "last_conversion_mean",
    "last_three_conversion_mean",
    "last_bid_success",
    "last_three_bid_success_mean",
    "historical_successful_bid_position_mean",
    "last_successful_bid_position_mean",
    "last_three_successful_bid_position_mean",
    "historical_cost_mean",
    "last_cost_mean",
    "last_three_cost_mean",
    "historical_cost_slot_1_mean",
    "last_cost_slot_1_mean",
    "last_three_cost_slot_1_mean",
    "historical_cost_slot_2_mean",
    "last_cost_slot_2_mean",
    "last_three_cost_slot_2_mean",
    "historical_cost_slot_3_mean",
    "last_cost_slot_3_mean",
    "last_three_cost_slot_3_mean",
    "historical_bid_over_lwc_mean",
    "last_bid_over_lwc_mean",
    "last_three_bid_over_lwc_mean",
    "historical_pv_over_lwc_mean",
    "last_pv_over_lwc_mean",
    "last_three_pv_over_lwc_mean",
    "historical_pv_over_lwc_90_pct",
    "last_pv_over_lwc_90_pct",
    "last_three_pv_over_lwc_90_pct",
    "historical_pv_over_lwc_99_pct",
    "last_pv_over_lwc_99_pct",
    "last_three_pv_over_lwc_99_pct",
    "current_pvalues_mean",
    "current_pvalues_90_pct",
    "current_pvalues_99_pct",
    "current_pv_num",
    "last_pv_num",
    "last_three_pv_num",
    "pv_num_total"
]

env = BiddingEnv(pvalues_path, bids_path, budget_range, target_cpa_range, obs_keys=keys_list, seed=seed)

Loading pvalues from /home/ubuntu/Dev/NeurIPS_Auto_Bidding_General_Track_Baseline/data/online_rl_data/period-7_pvalues.parquet
Loading bids from /home/ubuntu/Dev/NeurIPS_Auto_Bidding_General_Track_Baseline/data/online_rl_data/period-7_bids.parquet


In [15]:
len(keys_list)

60

In [7]:
advertiser = 8
delivery_period = 7
budget = training_data[(training_data["advertiserNumber"] == advertiser) & (training_data["deliveryPeriodIndex"] == delivery_period)]["budget"].mean()
target_cpa = training_data[(training_data["advertiserNumber"] == advertiser) & (training_data["deliveryPeriodIndex"] == delivery_period)]["CPAConstraint"].mean()
print(budget, target_cpa)

1500.0 7.0


In [8]:
state, _ = env.reset(budget, target_cpa, advertiser, delivery_period)
ts = 0


In [13]:
action = training_data[(training_data["advertiserNumber"] == advertiser) & (training_data["deliveryPeriodIndex"] == delivery_period) & (training_data["timeStepIndex"] == ts)]["action"].item()
action = action * 1.001  # Otherwise it ties whith itself when it made the third bid
state, reward, truncated, terminated, info = env.step(action / target_cpa)
ts += 1
print(ts)

5


In [14]:
pvalues, _ = env.get_pvalues_mean_and_std()
state_dict = env.get_state_dict(pvalues)
state_dict

{'time_left': 0.8958333333333334,
 'budget_left': 0.8801478531956672,
 'budget': 1500.0,
 'cpa': 7.0,
 'category': 1.0,
 'historical_bid_mean': 0.08966984084965748,
 'last_bid_mean': 0.09164262177294441,
 'last_three_bid_mean': 0.09323720167835671,
 'least_winning_cost_mean': 0.14380705495280216,
 'last_least_winning_cost_mean': 0.13231172224786397,
 'last_three_least_winning_cost_mean': 0.13736090556330902,
 'least_winning_cost_10_pct': 0.11766284629702568,
 'last_least_winning_cost_10_pct': 0.10989895537495613,
 'last_three_least_winning_cost_10_pct': 0.11299687102437019,
 'least_winning_cost_01_pct': 0.10134173540771008,
 'last_least_winning_cost_01_pct': 0.09633569970726967,
 'last_three_least_winning_cost_01_pct': 0.09803702875971794,
 'pvalues_mean': 0.00510308291193068,
 'conversion_mean': 0.0001654772194209814,
 'bid_success_mean': 0.06947536559736261,
 'last_pvalues_mean': 0.0050458038831811235,
 'last_three_pvalues_mean': 0.005100228033265007,
 'last_conversion_mean': 0.0,
 '

In [ ]:
state

In [ ]:
state = training_data[(training_data["advertiserNumber"] == advertiser) & (training_data["deliveryPeriodIndex"] == delivery_period) & (training_data["timeStepIndex"] == ts)]["state"].item()
state

In [ ]:
df = env.pvalues_df
ts_df = df[(df["advertiserNumber"] == advertiser) & (df["timeStepIndex"] == ts)]
pv = ts_df.pValue.item()
bids = action * pv

In [ ]:
raw_data_path = ROOT_DIR / "data" / "raw_traffic" / "period-7.csv"
raw_df = pd.read_csv(raw_data_path)

In [ ]:
prev_ts_df = raw_df[(raw_df["advertiserNumber"] == advertiser) & (raw_df["timeStepIndex"] == ts - 1)]
raw_pv = prev_ts_df.pValue.to_numpy()
raw_bids = action * raw_pv

In [ ]:
prev_ts_df.xi.mean()

In [ ]:
raw_min_cost = prev_ts_df.leastWinningCost
raw_xi = raw_bids > raw_min_cost
strange_df = prev_ts_df[(raw_xi) & (~prev_ts_df.xi.apply(bool))]

In [ ]:
raw_bids.shape

In [ ]:
prev_ts_df[raw_bids > prev_ts_df.bid]

In [ ]:
strange_df.bid > strange_df.leastWinningCost

In [ ]:
raw_df[(raw_df["timeStepIndex"] == ts - 1) & (raw_df.pvIndex == 1287)].sort_values("bid", ascending=False)

In [ ]:
raw_df.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 0.01, 100)
y = np.sqrt(x)
# plt.plot(x, y)
plt.plot(x, x)
plt.plot(x, 1. * x + 0.0 * y - 10 * x **2)